In [ ]:
!pip3 install virtualenv
!virtualenv -p python3.8.10 python38

!source /content/drive/MyDrive/Colab/RL_NLP/python38/bin/activate
!python --version

In [ ]:
from google.colab import drive
from os.path import join  
import sys

def mount_drive(ROOT):
    drive.mount(ROOT, force_remount=True)

ROOT = '/content/drive'
mount_drive(ROOT)

MY_GOOGLE_DRIVE_PATH = 'MyDrive/RL_NLP' 
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)   
%cd "{PROJECT_PATH}"
sys.path.insert(0, PROJECT_PATH)

In [ ]:
!pip install -r "{PROJECT_PATH}/metadata/requirements.txt"

### RUN IF GYM IS NOT THERE IN DRIVE.

!git clone https://github.com/Farama-Foundation/gym-examples

%cd gym-examples

!pip install -e .

%cd ..

# RUN FROM HERE.

In [ ]:
import numpy as np 
import pandas as pd 
import time
import utils as U
import model as M
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt 
import json
import copy

In [ ]:
!pip install -e .

In [ ]:
import gym
%cd gym-examples
import gym_examples
%cd ..
import numpy as np

episodes=2
env = gym.make('gym_examples/RlNlpWorld-v0',render_mode="rgb_array")

### Random Policy.

In [ ]:
def policy(observation):
    return np.random.randint(0,6)

dbg=True
episodes=1
env = gym.make('gym_examples/RlNlpWorld-v0',render_mode="rgb_array")
for _ in range(episodes):
    cumulative_reward,steps=0,0
    observation = env.reset(seed=42)
    cnt,mx_iter=0,10
    while mx_iter<steps:
        action = policy(observation)  # User-defined policy function
        observation, reward, terminated, info = env.step(action)
        cumulative_reward+=reward
        steps+=1
        if dbg==True:
            print(f'cumulative_reward {cumulative_reward}; action {action}')
        if terminated:
            break
    print(f'Cumulative Reward ~ {cumulative_reward}; TimeTaken ~ {steps}')
env.close()

### Delete Files.

In [ ]:
import os

def delete(state):
    file_path=f'/content/drive/MyDrive/RL_NLP/save_state/{state}.json'
    try:
        U.delete(state)
        os.unlink(file_path)
    except Exception as e:
        print(f'Error deleting {file_path}: {e}')

### Q-Learning w/o NLP. (NAIVE MODEL)

In [ ]:
!rm -rf "{PROJECT_PATH}/save_state"
!mkdir "{PROJECT_PATH}/save_state"
!mkdir "{PROJECT_PATH}/results"

In [ ]:
env = gym.make('gym_examples/RlNlpWorld-v0',mx_timeSteps=100,render_mode="rgb_array")
num_actions=6
update_after_actions=2**8
update_target_network=2**10
frame_cnt,epsilon_greedy=0,20000
episodes,mxSteps=100000,100
episode_reward_hist,episode_avg_reward=[],[]
epsilon,epsilon_min,epsilon_max=1.0,0.005,1.0 
epsilon_interval = (
    epsilon_max - epsilon_min
) 
epsilon_greedy_frames = 100000.0
batch_size=2**7
action=-1
replay_mem={'state':[],'next_state':[],'action':[],'reward':[],'small_n_state':[]}
mx_replay_memory=100000
gamma=0.9
# MODEL
save_step=5000;load_from_checkpoint=False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=10
first=True

In [ ]:
episodes=1000
load_from_checkpoint=True

#### Run this cell multiple times in short episodes because google colab throws error (disconnects with drive) if run for long.
Keep the cnt here -> 1

In [ ]:
model=M.NNModel().to(device)
model_traget=M.NNModel().to(device)
if load_from_checkpoint==True:
    model.load_state_dict(torch.load('results/model.ml'))
    model_traget.load_state_dict(torch.load('results/model.ml'))
    episode_reward_hist=np.load(f'results/ep_tot.npy').tolist()
    episode_avg_reward=np.load(f'results/ep_avg.npy').tolist()
    replay_mem['state']=np.load(f'results/state.npy').tolist()
    replay_mem['next_state']=np.load(f'results/next_state.npy').tolist()
    # replay_mem['small_n_state']=np.load(f'results/small_n_state.npy').tolist()
    replay_mem['action']=np.load(f'results/action.npy').tolist()
    replay_mem['reward']=np.load(f'results/reward.npy').tolist()
    file_read=open('results/frame_cnt.dat','r')
    frame_cnt=int(file_read.read())
    file_read.close()
    file_read=open('results/epsilon.dat','r')
    epsilon=float(file_read.read())
    file_read.close()
if first:   
    load_from_checkpoint=True
    first=False
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)
# frame_cnt
# epsilon
for _ in range(episodes):
    episode_reward=0
    state = env.reset(seed=42)
    for __ in range(mxSteps):
        frame_cnt+=1
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)
        U.phi(state);replay_mem['state'].append(U.save(state))
        if frame_cnt<=epsilon_greedy or epsilon>np.random.rand(1)[0]:
            action=np.random.choice(num_actions)
            # with open('my_dict.json', 'w') as f:
            #     U.phi(state)
            #     state["visual"]=state["visual"].tolist()
            #     json.dump(state,f)
        else:
            action=(M.predict(model,[U.get(replay_mem['state'][-1])],device))[1]
            action=action.item()
        next_state,reward,terminated,info=env.step(action)
        episode_reward+=reward 
        # temp_next_state=copy.deepcopy(next_state) #to save small image.
        state=copy.deepcopy(next_state)
        U.phi(next_state);replay_mem['next_state'].append(U.save(next_state))
        # U.phiXtra(temp_next_state);replay_mem['small_n_state'].append(U.save(temp_next_state))
        replay_mem['action'].append(action)
        replay_mem['reward'].append(reward)
        if (frame_cnt%save_step==0): # SAVE model temporarily so progress is not lost.
            print(f'Saving ...')
            np.save(f'results/ep_tot.npy',np.array(episode_reward_hist))
            np.save(f'results/ep_avg.npy',np.array(episode_avg_reward))
            np.save(f'results/state.npy',np.array(replay_mem['state']))
            np.save(f'results/next_state.npy',np.array(replay_mem['next_state']))
            # np.save(f'results/small_n_state.npy',np.array(replay_mem['small_n_state']))
            np.save(f'results/action.npy',np.array(replay_mem['action']))
            np.save(f'results/reward.npy',np.array(replay_mem['reward']))
            file_write=open('results/frame_cnt.dat','w')
            file_write.write(str(frame_cnt))
            file_write.close()
            file_write=open('results/epsilon.dat','w')
            file_write.write(str(epsilon))
            file_write.close()
            torch.save(model.state_dict(),'results/model.ml')
        if frame_cnt%update_after_actions==0 and len(replay_mem['reward'])>batch_size:
            choices=np.random.choice( range( len(replay_mem['reward']) ), size=batch_size )
            STATE=[U.get(replay_mem['state'][id]) for id in choices]
            NEXT_STATE=[U.get(replay_mem['next_state'][id]) for id in choices]
            # SMALL_NEXT_STATE=[U.get(replay_mem['small_n_state'][id]) for id in choices]
            ACTION=torch.stack([U.oneHot(num_actions,replay_mem['action'][id]).squeeze() for id in choices])
            REWARD=np.array([replay_mem['reward'][id] for id in choices])
            IMG=np.array([_state_["visual"] for _state_ in STATE])
            TEMP=M.predict(model_traget,NEXT_STATE,device)
            REWARD_T=torch.tensor(REWARD)
            if torch.cuda.is_available():
                REWARD_T=REWARD_T.to(torch.device("cuda:0")) 
            reward_true=torch.add(REWARD_T,gamma*TEMP[0][0])
            for epoch in range(epochs):
                # M.train(model,reward_true,STATE,SMALL_NEXT_STATE,ACTION,device,optimizer,type='dqn')
                # M.train(model,reward_true,STATE,SMALL_NEXT_STATE,ACTION,device,optimizer,type='image')
                M.train(model,reward_true,STATE,ACTION,device,optimizer,type='dqn')
                M.train(model,reward_true,STATE,ACTION,device,optimizer,type='image')
        if frame_cnt%update_target_network==0:
            print(f'Updating N/W ...{frame_cnt}')
            model_traget.load_state_dict(model.state_dict())
        if len(replay_mem['reward'])>mx_replay_memory:
            print('Deleting ...')
            delete(replay_mem['state'][:1][0]);del replay_mem['state'][:1]
            # delete(replay_mem['small_n_state'][:1][0]);del replay_mem['small_n_state'][:1]
            delete(replay_mem['next_state'][:1][0]);del replay_mem['next_state'][:1]
            del replay_mem['action'][:1]
            del replay_mem['reward'][:1]
        if terminated:
            break
    episode_reward_hist.append(episode_reward)
    episode_avg_reward.append(episode_reward/__)

# Save Relevant Results
np.save('results/ep_tot.npy',np.array(episode_reward_hist))
np.save('results/ep_avg.npy',np.array(episode_avg_reward))
torch.save(model.state_dict(),'results/model.ml')

In [ ]:
# Loading Results.
episode_reward_hist=np.load('results/ep_tot.npy')
episode_avg_reward=np.load('results/ep_avg.npy')
# U.plot(episode_reward_hist,'Total Reward','Total Reward V/S Episodes')
# U.plot(episode_avg_reward,'Avg. Reward','Avg. Reward Per Step V/S Episodes')
model_tr=M.NNModel().to(device)
model_tr.load_state_dict(torch.load('results/model.ml'))

In [ ]:
plt.plot(episode_reward_hist)

### Q-Learning with NLP Fully.

In [ ]:
!rm -rf "{PROJECT_PATH}/save_state"
!mkdir "{PROJECT_PATH}/save_state"
!mkdir "{PROJECT_PATH}/results"

In [ ]:
import model_nlp as M_NLP

In [ ]:
env = gym.make('gym_examples/RlNlpWorld-v0',mx_timeSteps=50,render_mode="rgb_array")
num_actions=6
update_after_actions=2**6
update_target_network=2**8
frame_cnt,epsilon_greedy=0,10000
episodes,mxSteps=100000,100
episode_reward_hist,episode_avg_reward=[],[]
epsilon,epsilon_min,epsilon_max=1.0,0.005,1.0 
epsilon_interval = (
    epsilon_max - epsilon_min
) 
epsilon_greedy_frames = 10000.0
batch_size=2**7
action=-1
replay_mem={'state':[],'next_state':[],'action':[],'reward':[],'small_n_state':[]}
mx_replay_memory=10000
gamma=0.9
# MODEL
save_step=5000;load_from_checkpoint=False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=10
first=True

In [ ]:
episodes=1000
load_from_checkpoint=True

In [ ]:
model=M_NLP.NNModelNLP().to(device)
model_traget=M_NLP.NNModelNLP().to(device)
if load_from_checkpoint==True:
    model.load_state_dict(torch.load('results/model_nlpf.ml'))
    model_traget.load_state_dict(torch.load('results/model_nlpf.ml'))
    episode_reward_hist=np.load(f'results/ep_tot_nlpf.npy').tolist()
    episode_avg_reward=np.load(f'results/ep_avg_nlpf.npy').tolist()
    replay_mem['state']=np.load(f'results/state_nlpf.npy').tolist()
    replay_mem['next_state']=np.load(f'results/next_state_nlpf.npy').tolist()
    replay_mem['action']=np.load(f'results/action_nlpf.npy').tolist()
    replay_mem['reward']=np.load(f'results/reward_nlpf.npy').tolist()
    file_read=open('results/frame_cnt_nlpf.dat','r')
    frame_cnt=int(file_read.read())
    file_read.close()
    file_read=open('results/epsilon_nlpf.dat','r')
    epsilon=float(file_read.read())
    file_read.close()
if first:   
    load_from_checkpoint=True
    first=False
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)
try:
    for _ in range(episodes):
        episode_reward=0
        state = env.reset(seed=42)
        print(f'Episode no {_}')
        for __ in range(mxSteps):
            frame_cnt+=1
            epsilon -= epsilon_interval / epsilon_greedy_frames
            epsilon = max(epsilon, epsilon_min)
            U.phi(state);replay_mem['state'].append(U.save(state))
            if frame_cnt<=epsilon_greedy or epsilon>np.random.rand(1)[0]:
                action=np.random.choice(num_actions)
            else:
                action=(M_NLP.predict(model,[U.get(replay_mem['state'][-1])],device))[1]
                action=action.item()
            next_state,reward,terminated,info=env.step(action)
            episode_reward+=reward 
            # temp_next_state=copy.deepcopy(next_state) #to save small image.
            state=copy.deepcopy(next_state)
            U.phi(next_state);replay_mem['next_state'].append(U.save(next_state))
            # U.phiXtra(temp_next_state);replay_mem['small_n_state'].append(U.save(temp_next_state))
            replay_mem['action'].append(action)
            replay_mem['reward'].append(reward)
            if frame_cnt%update_after_actions==0 and len(replay_mem['reward'])>batch_size:
                choices=np.random.choice( range( len(replay_mem['reward']) ), size=batch_size )
                STATE=[U.get(replay_mem['state'][id]) for id in choices]
                NEXT_STATE=[U.get(replay_mem['next_state'][id]) for id in choices]
                # SMALL_NEXT_STATE=[U.get(replay_mem['small_n_state'][id]) for id in choices]
                ACTION=torch.stack([U.oneHot(num_actions,replay_mem['action'][id]).squeeze() for id in choices])
                REWARD=np.array([replay_mem['reward'][id] for id in choices])
                IMG=np.array([_state_["visual"] for _state_ in STATE])
                TEMP=M_NLP.predict(model_traget,NEXT_STATE,device)
                REWARD_T=torch.tensor(REWARD)
                if torch.cuda.is_available():
                    REWARD_T=REWARD_T.to(torch.device("cuda:0")) 
                reward_true=torch.add(REWARD_T,gamma*TEMP[0][0])
                for epoch in range(epochs):
                    # M_NLP.train(model,reward_true,STATE,SMALL_NEXT_STATE,ACTION,device,optimizer,type='dqn')
                    # M_NLP.train(model,reward_true,STATE,SMALL_NEXT_STATE,ACTION,device,optimizer,type='image')
                    M_NLP.train(model,reward_true,STATE,ACTION,device,optimizer,type='dqn')
                    M_NLP.train(model,reward_true,STATE,ACTION,device,optimizer,type='image')
            if frame_cnt%update_target_network==0:
                model_traget.load_state_dict(model.state_dict())
            if len(replay_mem['reward'])>mx_replay_memory:
                delete(replay_mem['state'][:1][0]);del replay_mem['state'][:1]
                # delete(replay_mem['small_n_state'][:1][0]);del replay_mem['small_n_state'][:1]
                delete(replay_mem['next_state'][:1][0]);del replay_mem['next_state'][:1]
                del replay_mem['action'][:1]
                del replay_mem['reward'][:1]
            if terminated:
                break
        episode_reward_hist.append(episode_reward)
        episode_avg_reward.append(episode_reward/__)
    # Save Relevant Results
    np.save('results/ep_tot_nlpf.npy',np.array(episode_reward_hist))
    np.save('results/ep_avg_nlpf.npy',np.array(episode_avg_reward))
    torch.save(model.state_dict(),'results/model_nlpf.ml')
except Exception as e:
    print(f'Saving ...')
    np.save(f'results/ep_tot_nlpf.npy',np.array(episode_reward_hist))
    np.save(f'results/ep_avg_nlpf.npy',np.array(episode_avg_reward))
    np.save(f'results/state_nlpf.npy',np.array(replay_mem['state']))
    np.save(f'results/next_state_nlpf.npy',np.array(replay_mem['next_state']))
    np.save(f'results/action_nlpf.npy',np.array(replay_mem['action']))
    np.save(f'results/reward_nlpf.npy',np.array(replay_mem['reward']))
    file_write=open('results/frame_cnt_nlpf.dat','w')
    file_write.write(str(frame_cnt))
    file_write.close()
    file_write=open('results/epsilon_nlpf.dat','w')
    file_write.write(str(epsilon))
    file_write.close()
    torch.save(model.state_dict(),'results/model_nlpf.ml')
    print(e)

### Q-Learning with NLP. {TBD}

In [ ]:
num_actions=6
update_after_actions=2**8
update_target_network=2**10
frame_cnt,epsilon_greedy=0,20000
episodes,mxSteps=100000,100
episode_reward_hist,episode_avg_reward=[],[]
epsilon,epsilon_min,epsilon_max=1.0,0.005,1.0 
epsilon_interval = (
    epsilon_max - epsilon_min
) 
epsilon_greedy_frames = 100000.0
batch_size=2**7
action=-1
replay_mem={'state':[],'next_state':[],'action':[],'reward':[],'small_n_state':[]}
mx_replay_memory=100000
gamma=0.9
# MODEL
save_step=5000;load_from_checkpoint=False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=10
first=True

In [ ]:
episodes=1000
load_from_checkpoint=False

In [ ]:
model=M_NLP.NNModelNLP().to(device)
model_traget=M_NLP.NNModelNLP().to(device)
if load_from_checkpoint==True:
    model.load_state_dict(torch.load('results/model_nlp.ml'))
    model_traget.load_state_dict(torch.load('results/model_nlp.ml'))
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)
epochs=10
#### REMOVE INSTRUCTIONS SLOWLY MODEL IS NOT SO DEPENDENT ####
epsilon_nlp=1.0
epsilon_greedy_frames_nlp=100000.0
##############################################################

for _ in range(episodes):
    episode_reward=0
    state = env.reset(seed=42)
    for __ in range(mxSteps):
        frame_cnt+=1
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)
#### REMOVE INSTRUCTIONS SLOWLY MODEL IS NOT SO DEPENDENT ####
        epsilon_nlp -= epsilon_interval / epsilon_greedy_frames_nlp
        if epsilon_nlp<np.random.rand(1)[0]:
            state['text']='[PAD]'
##############################################################
        U.phi(state);replay_mem['state'].append(U.save(state))
        if frame_cnt<=epsilon_greedy or epsilon>np.random.rand(1)[0]:
            action=np.random.choice(num_actions)
        else:
            action=(M_NLP.predict(model,[U.get(replay_mem['state'][-1])],device))[1]
            action=action.item()
        next_state,reward,terminated,info=env.step(action)
        episode_reward+=reward 
        temp_next_state=copy.deepcopy(next_state) #to save small image.
        state=copy.deepcopy(next_state)
        U.phi(next_state);replay_mem['next_state'].append(U.save(next_state))
        U.phiXtra(temp_next_state);replay_mem['small_n_state'].append(U.save(temp_next_state))
        replay_mem['action'].append(action)
        replay_mem['reward'].append(reward)
        if (_!=0 and _%save_step==0): # SAVE model temporarily so progress is not lost.
            np.save(f'results/ep_tot_nlp{_}.npy',np.array(episode_reward_hist))
            np.save(f'results/ep_avg_nlp{_}.npy',np.array(episode_avg_reward))
            torch.save(model.state_dict(),'results/model_nlp.ml')
        if frame_cnt%update_after_actions==0 and len(replay_mem['reward'])>batch_size:
            choices=np.random.choice( range( len(replay_mem['reward']) ), size=batch_size )
            STATE=[U.get(replay_mem['state'][id]) for id in choices]
            NEXT_STATE=[U.get(replay_mem['next_state'][id]) for id in choices]
            SMALL_NEXT_STATE=[U.get(replay_mem['small_n_state'][id]) for id in choices]
            ACTION=torch.stack([U.oneHot(num_actions,replay_mem['action'][id]).squeeze() for id in choices])
            REWARD=np.array([replay_mem['reward'][id] for id in choices])
            IMG=np.array([_state_["visual"] for _state_ in STATE])
            TEMP=M_NLP.predict(model_traget,NEXT_STATE,device);
            REWARD_T=torch.tensor(REWARD)
            reward_true=torch.add(REWARD_T,gamma*TEMP[0][0])
            for epoch in range(epochs):
                M_NLP.train(model,reward_true,STATE,SMALL_NEXT_STATE,ACTION,device,optimizer,type='dqn')
                M_NLP.train(model,reward_true,STATE,SMALL_NEXT_STATE,ACTION,device,optimizer,type='image')
        if frame_cnt%update_target_network==0:
            model_traget.load_state_dict(model.state_dict())
        if len(replay_mem['reward'])>mx_replay_memory:
            delete(replay_mem['state'][:1][0]);del replay_mem['state'][:1]
            delete(replay_mem['small_n_state'][:1][0]);del replay_mem['small_n_state'][:1]
            delete(replay_mem['next_state'][:1][0]);del replay_mem['next_state'][:1]
            del replay_mem['action'][:1]
            del replay_mem['reward'][:1]
        if terminated:
            break
    
    episode_reward_hist.append(episode_reward)
    episode_avg_reward.append(episode_reward/__)

# Save Relevant Results
np.save('results/ep_tot_nlp.npy',np.array(episode_reward_hist))
np.save('results/ep_avg_nlp.npy',np.array(episode_avg_reward))
torch.save(model.state_dict(),'results/model_nlp.ml')